# Otimização

Em um sentido mais amplo, a otimização consiste em buscar a melhor solução possível para um determinado problema.

Em diversos problemas de engenharia a otimização inicia pelo estágio da criação de um modelo matemático do sistema que deseja-se otimizar e, em seguida, a aplicação de técnicas matemáticas para maximizar ou minimizar uma função que caracteriza o objetivo ou o custo.

O problema de otimização pode ser mais formalmente definido como:

\begin{equation} 
minimizar f(x), x \in \Omega
\end{equation}

Outra formulação comum é quando queremos achar o máximo de uma função, ou então:

\begin{equation} 
maximizar f(x), x \in \Omega
\end{equation}

O caso mais simples consiste quando a função que deseja-se otimizar é unidimensional.
Neste caso, temos que $x \in \mathbb{R}$

## Exemplo


Por exemplo, suponha que desejamos projetar um recipiente cilíndrico capaz de armazenar 1L de volume.
Desejamos otimizar o custo de produção do recipiente.

Podemos simplificar o problema e associar o custo de produção à quantidade de material utilizado.
Desta forma, o problema transforma-se em saber qual o raio do cilindro que minimiza a quantidade de material utilizada.

Sabendo que o volume é dado por:

\begin{equation} \label{eq:otimizacao1}
V = h\pi r^2
\end{equation}

enquanto que a área é dado pela soma da área da base, do topo e da lateral:

$
A(r,h) = \pi r^2 + \pi r^2 + 2\pi r h
$

Então, substituindo o volume desejado na equação \ref{eq:otimizacao1} podemos isolar o valor de $h$ e encontrar a equação:

$
A(r) = 2\pi r^2 + 2/r
$

Chegamos então a formulação do nosso problema:


\begin{matrix}
\min & A(r), r \in \mathbb{R}^+
\end{matrix}


Podemos então utilizar a função `minimize` para determinar o menor raio:

In [1]:
import scipy.optimize as opt
from math import pi

# funcao que calcula a area, baseado no raio
def A(r):
    return 2*pi*r**2 +2/r

opt.minimize(A, 0.1)

      fun: 5.535810445932086
 hess_inv: array([[0.02653506]])
      jac: array([5.96046448e-08])
  message: 'Optimization terminated successfully.'
     nfev: 22
      nit: 8
     njev: 11
   status: 0
  success: True
        x: array([0.54192606])

In [10]:
import pandas as pd
import plotly.express as pe
import plotly.io as pio
import numpy as np

pio.renderers.default = "plotly_mimetype+notebook_connected"

X = np.linspace(0.1, 2.0, 200)
Y = A(X)

data = pd.DataFrame(list(zip(X,Y)), columns = ['raio', 'area'])

fig = pe.line(data, x='raio', y='area', width=800, 
        template='plotly_white', title="Área do cilindro x raio")
fig.update_layout( xaxis=dict( rangeslider=dict( visible=True ), type="linear" ) )
fig.show()

## Otimização - derivada

Lembre-se do cálculo que os pontos de elevação máxima e mínima correspondem, matematicamente, aos pontos onde a derivada da função é igual a zero.

Desta forma, uma primeira abordagem baseado no cálculo é  calcular analiticamente a derivada da função que desejamos minimizar (ou maximizar) e, em seguida, determinar os zeros da derivada.

### Exemplo

Suponha que em uma solução aquosa encontram-se dissolvidos íons $H^+$ e $OH^-$, de forma que suas concentrações obedeçam à relação $\left[ H^+ \right] \left[ OH^- \right] = 10^{-14}$.

Determine qual a concentração de $H^+$ que minimiza a soma de $H^+ + OH^-$.

A função que queremos minimizar é:

\begin{equation}
f(H^+,OH^-) = \left[H^+\right] + \left[OH^-\right]
\end{equation}

A partir da restrição fornecida, podemos substituir uma das concentrações e simplificar o problema para:

\begin{equation}
f(H^+) = \left[H^+\right] + \dfrac{10^{-14}}{\left[H^+\right]}
\end{equation}

cuja derivada pode ser facilmente obtida em:

\begin{equation}
f'(H^+) = 1 - \dfrac{10^{-14}}{\left[H^+\right]^2}
\end{equation}

Neste ponto, podemos utilizar qualquer uma das técnicas para encontrar o zero desta função.

Por exemplo, resolvendo analiticamente obtemos o valor:

$
\left[H^+\right] = 10^{-7}
$

## Otimização - abordagem numérica

Da mesma forma que vimos no problema de determinação de raízes, podemos encontrar um problema onde não seja possível determinar analiticamente a derivada de uma função.

Neste caso, podemos utilizar um método numérico para determinar o máximo ou mínimo de uma função.

No **método da razão áurea**, vamos, similarmente ao que foi feito na bissecção, buscar um subintervalo no qual o ótimo da função deve se encontrar.
Supomos também que só existe um único ótimo no intervalo $(x_l, x_u)$

Entretanto, ao invés de utilizarmos apenas 1 ponto, vamos utilizar 2 pontos interiores, calculados conforme seguem:

\begin{equation}
x_1 = x_l + d
\end{equation}

\begin{equation}
x_2 = x_u - d
\end{equation}

onde

\begin{equation}
d = (\phi - 1)(x_u - x_l)
\end{equation}

e $\phi = \frac{1+\sqrt{5}}{2}$ é a razão áurea, originalmente descrita por Euclides. 

Em seguida, compara-se o valor da função nestes dois pontos determinando-se assim em qual subinvervalo o mínimo deve estar localizado.

O teste a ser realizado é:

se $f(x_1) < f(x_2)$ então deve-se buscar no intervalo da direita, definido por $(x_2, x_u)$.

se $f(x_2) < f(x_1)$ então deve-se buscar no intervalo da esquerda, definido por $(x_l, x_1)$.


<div>
<img src="figuras/otimizacao_razaoaurea.svg" width="400"/>
</div>

In [29]:
from numpy import sin
import math as m

def funcao(x):
    return x**2/10 - 2*sin(x)

def otimizacao_aurea_ingenua(f, xl, xu, eps):
    erro = abs(xu - xl)/2
    phi = (1+m.sqrt(5))/2
    while erro > eps:
        d = (phi - 1)*(xu - xl)
        x1 = xl + d
        x2 = xu - d
        if f(x1) < f(x2):
            xl = x2
            xmin = x2
        else:
            xu = x1
            xmin = x1
        erro = abs(xu - xl)/2
        print(erro)
    return xmin

In [30]:
otimizacao_aurea_ingenua(funcao, 0, 4, 0.001)

1.2360679774997898
0.7639320225002104
0.4721359549995795
0.29179606750063103
0.18033988749894858
0.11145618000168256
0.06888370749726613
0.042572472504416425
0.026311234992849708
0.016261237511566717
0.01004999748128299
0.006211240030283727
0.003838757450999264
0.002372482579284463
0.001466274871714801
0.0009062077075696617


1.4266717891007419

In [41]:
import scipy.optimize as opt

sol = opt.minimize_scalar(funcao, bracket=[0,4])
print('Minimize scalar:\n',sol)

sol = opt.minimize(funcao, 0)
print('\nMinimize:\n', sol)

Minimize scalar:
      fun: -1.7757256531474153
    nfev: 14
     nit: 10
 success: True
       x: 1.427551781310228

Minimize:
       fun: -1.775725653147415
 hess_inv: array([[0.45885959]])
      jac: array([0.])
  message: 'Optimization terminated successfully.'
     nfev: 14
      nit: 6
     njev: 7
   status: 0
  success: True
        x: array([1.42755177])


## Programação linear

Uma formulação comum nos problemas de otimização é chamado de programação linear.

Nestes casos, o problema é sujeito a restrições lineares, tal que ele pode ser descrito como:

\begin{array}{ll@{}}
min & \sum c_i x_i \\
\text{sujeito à:} & A_lx \leq b_l \\
& A_ex = b_e \\
& l \leq x \leq u
\end{array}

onde 
$\sum c_i x_i$ é a chamada função objetivo que desejamos minimizar, $x_i$ é o vetor contendo as variáveis de decisão, $A_e$, $b_e$, $A_l$, $b_l$, $l$ e $u$ são matrizes ou vetores que modelam as restrições lineares do problema.

Por exemplo, suponha que um sítio precise alimentar suas galinhas.
Ela pode utilizar 3 tipos de ração (A, B e C), e o custo de cada um é R\\$6,00 e R\\$9,00  e R\\$12,00 respectivamente.
As galinhas necessitam de uma porção mínima de 3g de proteína e 5g de cálcio.
A ração A provê 0,2g/un de proteína e 0,4g/un de cálcio, a ração B provê 0,6 de proteína e 0,6 de cálcio, enquanto que a ração C provê 0,7 de proteína e 0,8 de cálcio.

Qual a melhor forma de misturar as rações para atingir o balanço nutricional desejado com menor custo possível?

As variáveis de decisão serão: 

* $x_1$: quantidade de ração A
* $x_2$: quantidade de ração B
* $x_3$: quantidade de ração C

E o problema pode ser formulado como:

\begin{array}{ll@{}}
min &
\begin{bmatrix}
6 & 9 & 12
\end{bmatrix} 
\begin{bmatrix}
x_1 \\ x_2 \\ x_3
\end{bmatrix} 
\\
\text{sujeito à:} & 
\begin{bmatrix}
0,2 & 0,6 & 0,7 \\
0,4 & 0,6 & 0,8
\end{bmatrix}
\begin{bmatrix}
x_1 \\ x_2 \\ x_3
\end{bmatrix}
\leq 
\begin{bmatrix}
3 \\ 5
\end{bmatrix}
\end{array}


In [38]:
import scipy.optimize as opt

c = [6, 9, 12]
Al = -1.*np.array([[0.2, 0.6, 0.7], [0.4, 0.6, 0.8]])
bl = -1.*np.array([3, 5])

sol = opt.linprog(c, Al, bl)
print(sol)

     con: array([], dtype=float64)
     fun: 74.99999998494948
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([ 1.22926938e+00, -1.00336717e-09])
  status: 0
 success: True
       x: array([2.47516373, 3.00725248, 2.75697877])


## Gradiente

Existem diversas formas de resolver o problema de otimização no caso de funções multidimensionais.

Uma família delas baseia-se na derivada da função e, de forma parecida com o método de Newton, utilizamos a derivada multidimensional (mais conhecido como **gradiente**) para indicar a direção que deverá ser percorrida.

O algoritmo busca, em cada iteração, a direção com maior declividade, e caminha um passo nesta direção.
Em seguida, realiza os mesmos cálculos para o próximo passo.
O comportamento observado é o de uma esfera descendo por um vale.

Matematicamente, podemos definir o algoritmo da seguinte forma:

Dado uma função $f = f(x_1, x_2, \ldots, x_n)$, o gradiente dela é definido como:

\begin{equation}
\nabla f = \begin{bmatrix}
\dfrac{\partial f}{\partial x_1} \ldots \dfrac{\partial f}{\partial x_n} 
\end{bmatrix}
\end{equation}

O algoritmo baseado na descida do gradiente busca, em cada instante, caminhar um passo de tamanho $\gamma$ na direção de maior declividade, atualizando sua posição conforme a equação:

\begin{equation}
x_{i+1} = x_i - \gamma \nabla f(x_i)
\end{equation}

In [30]:
import scipy.optimize as opt

def f(X):
    # 2 + x1 - x2 +2x1^2 + 2x1x2 + x2^2
    return 2 + X[0] - X[1] + 2*X[0]**2 +2*X[0]*X[1] + X[1]**2
    
def jacobiano(X):
    # [1 + 4x1 +2x2 ; -1 + 2x1 + 2x2]
    j11 = 1 + 4*X[0] + 2*X[1]
    j12 = -1 +2*X[0] +2*X[1]
    return [j11, j12]

opt.minimize(f, x0=[0,0], jac=jacobiano)

      fun: 0.75
 hess_inv: array([[ 0.5, -0.5],
       [-0.5,  1. ]])
      jac: array([0., 0.])
  message: 'Optimization terminated successfully.'
     nfev: 6
      nit: 4
     njev: 6
   status: 0
  success: True
        x: array([-1. ,  1.5])

In [32]:
def otimizacao_gradiente(f, j, x0, step, n):
    xi = x0
    X = []
    Y = []
    for i in range(n):
        xi = xi - step*np.array(j(xi))
        X.append(xi[0])
        Y.append(xi[1])
    return xi, X, Y

x, Xgrad,Ygrad = otimizacao_gradiente(f,jacobiano, [-1.5,0.5], 0.1, 500)
print(x)

[-1.   1.5]


In [29]:
import plotly.graph_objects as go
import numpy as np

amostras = 20
XL = np.linspace(-2.0, 0.0, amostras)
YL = np.linspace(0.0, 3.0, amostras)

Z = np.empty([amostras,amostras])
for ix,x in enumerate(XL):
    for iy,y in enumerate(YL):
        Z[ix,iy] = f([x,y])

fig = go.Figure(go.Contour(z=Z, x= XL, y=YL, 
                         contours_coloring='lines', contours=dict(start=0, end=8, size=1)))

fig.add_trace(go.Scatter(x=Xgrad,y=Ygrad))

fig.update_layout(title_text='Curvas de nível', width=500, height=600, autosize=False)
fig.show()

In [11]:
import plotly.graph_objects as go
import numpy as np

amostras = 20
XL = np.linspace(-2.0, 0.0, amostras)
YL = np.linspace(0.0, 3.0, amostras)

Z = np.empty([amostras,amostras])
for ix,x in enumerate(XL):
    for iy,y in enumerate(YL):
        Z[ix,iy] = f([x,y])

fig = go.Figure(go.Surface(z=Z, x=XL, y=YL))

fig.update_layout(title_text='Superfície 3D', width=800, height=500, autosize=False,scene_aspectmode='cube')
fig.show()

## Exemplo 1

[CHAPRA, Ex. 7.26] A taxa de crescimento específico de uma cultura que produz um antibiótico é uma função da concentração de alimento $c$:

\begin{equation}
g = \dfrac{2c}{4 + 0,8c + c^2 +0,2c^3}
\end{equation}

O crescimento tende a zero em concentrações muito baixas em decorrência da limitação de alimentos, e também tende a zero em concentrações muito altas por causa do efeito de toxidade.
Encontre o valor no qual o crescimento é maximo.

<img src="figuras/otimizacao_ex7_26.svg" width="500"/>

## Exemplo 2

Um fazendeiro deseja cercar uma área de sua região.
Uma das laterais faz fronteira com um área previamente cercada, de forma que ele não precisará gastar material neste lado.
Supondo que ele possui 100m de cerca, quais as dimensões que maximizam a área cercada?